In [1]:
import pandas

In [2]:
tmp_data = pandas.read_csv("good.tsv", sep="\t")

In [3]:
tmp_data.head()

,context_2,context_1,context_0,reply,label
0,"кликни на меня а потом на надпись "" видео - зв...","о , я тебя вижу .","ладно , повесь трубку .",не могу .,good
1,"кликни на меня а потом на надпись "" видео - зв...","о , я тебя вижу .","ладно , повесь трубку .","нет , звонить буду я .",neutral
2,"кликни на меня а потом на надпись "" видео - зв...","о , я тебя вижу .","ладно , повесь трубку .",я не прекращу звонить .,good
3,"кликни на меня а потом на надпись "" видео - зв...","о , я тебя вижу .","ладно , повесь трубку .",я звоню им .,good
4,бывало и получше .,"слушайте , мы с женой . . . совсем заблудились .","я пытаюсь добраться до юджина , но , кажется ,...","куда же вы едете , месье ?",good


In [4]:
tmp_data.shape

(60049, 5)

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
vectorizer.fit(tmp_data.context_0)
words_amount_matrix = vectorizer.transform(tmp_data.context_0)

In [6]:
words_amount_matrix.shape

(60049, 14123)

In [7]:
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=300)
svd.fit(words_amount_matrix)
context_words_amount_matrix = svd.transform(words_amount_matrix)

In [8]:
context_words_amount_matrix.shape

(60049, 300)

## Основные функции

In [9]:
from sklearn.neighbors import BallTree
from sklearn.base import BaseEstimator
import numpy

In [10]:
def softmax(x):
    prob = numpy.exp(-x)
    return prob/sum(prob)

In [11]:
class NeighborChooser(BaseEstimator):
    def __init__(self, k=5, temperature=1.0):
        self.k = k
        self.temperature = temperature
    
    def fit(self, X, y):
        self.Tree = BallTree(X)
        self.y = numpy.array(y)
    
    def predict(self, X, rendom_state=None):
        distances, indexes = self.Tree.query(X, return_distance=True, k=self.k)
        result = []
        for dist, index in zip(distances, indexes):
            result.append(numpy.random.choice(index, p=softmax(dist*self.temperature)))
        return self.y[result]

In [12]:
from sklearn.pipeline import make_pipeline
neighborChooser = NeighborChooser()

In [13]:
neighborChooser.fit(context_words_amount_matrix, tmp_data.reply)
pipe = make_pipeline(vectorizer, svd, neighborChooser)

# ВК БОТ

In [36]:
import vk_api
from vk_api.longpoll import VkLongPoll, VkEventType
import random

In [ ]:
def write_msg(user_id, message):
    vk.method('messages.send', {'user_id': user_id, 'message': message, 'random_id': random.randint(0, 2048)})

# API-ключ созданный ранее
token = "9fc05183b65cc425a0e5d2b49ced1b778d41ade21d1827bf2fc9555a9417b6295433f6b6c6a3c0ec05fab"

# Авторизуемся как сообщество
vk = vk_api.VkApi(token=token)

# Работа с сообщениями
longpoll = VkLongPoll(vk)

# Основной цикл
for event in longpoll.listen():

    # Если пришло новое сообщение
    if event.type == VkEventType.MESSAGE_NEW:
    
        # Если оно имеет метку для меня( то есть бота)
        if event.to_me:
        
            # Сообщение от пользователя
            request = event.text
            
            if event.user_id == 302374904:
                rand_num = random.random()
                if rand_num <= 0.1:
                    write_msg(event.user_id, "Федя лох, хехе")
                else:
                    write_msg(event.user_id, pipe.predict([request]))
            else:
                write_msg(event.user_id, pipe.predict([request]))